connTask is a package used for the prediction of task evoked patterns of activity (derived from tFMRI data) using functional connectivity data

it is based on the work by Tavor et al (2016) https://science.sciencemag.org/content/352/6282/216



In [3]:
import utils, extract_features, model_and_predict

In [4]:
print('hey')

hey
